# Install Packages

In [ ]:
!pip install -q transformers accelerate peft bitsandbytes
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install sentence-transformers faiss-cpu
!pip install python-docx PyMuPDF!
!pip install PyPDF2
!pip install fastapi uvicorn nest-asyncio pyngrok
!pip install sqlalchemy pyodbc
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 77.6 MB/s eta 0:00:00
ERROR: Invalid requirement: 'PyMuPDF!': E

# Import Libraries

In [ ]:
import os
import io
import re
import gc
import sys
import json
import asyncio
import zipfile
import logging

import torch
import numpy as np

from io import BytesIO

from typing import List, Dict, Optional, Tuple, IO

from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

from PyPDF2 import PdfReader  # hoặc dùng PyPDF2.PdfReader nếu bạn chưa sửa
from sentence_transformers import SentenceTransformer, util

import google.generativeai as genai

from google.colab import drive
from google.colab import userdata

In [ ]:
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["NGROK_AUTH_TOKEN"] = userdata.get("NGROK_AUTH_TOKEN")

# Connect Drive

In [ ]:
#Mount Google Drive
drive.mount('/content/drive')

#Load Finetuned Model
zip_path = '/content/drive/My Drive/iso9001-fine-tuned-model.zip'
extract_dir = '/content/iso9001-fine-tuned-model'

if not os.path.exists(extract_dir):
    print(f"Extracting zip file to: {extract_dir}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Extraction completed.")
else:
    print(f"Directory already exists: {extract_dir}. Skipping extraction.")

Mounted at /content/drive
Extracting zip file to: /content/iso9001-fine-tuned-model
Extraction completed.


# Gemini Chatbot

In [ ]:
class ISO9001RAGChatbot:
    def __init__(self, gemini_api_key: str):
        genai.configure(api_key=gemini_api_key)
        self.model = genai.GenerativeModel("gemini-2.5-flash")

        self.clause_library = {
            "7.5": [
                {
                    "clause": "DOC-01",
                    "title": "Tiêu đề tài liệu",
                    "description": "Tài liệu phải có tiêu đề rõ ràng, phản ánh đúng nội dung và mục đích sử dụng.",
                    "example": "BÁO CÁO tổng kết hoạt động năm học 2023-2024",
                    "pattern_hint": r"(BÁO CÁO|QUY TRÌNH|KẾ HOẠCH|THÔNG BÁO)"
                },
                {
                    "clause": "DOC-02",
                    "title": "Mã tài liệu",
                    "description": "Tài liệu cần có mã hoặc số được định danh rõ ràng.",
                    "example": "Số: 531/BC-THTTHS",
                    "pattern_hint": r"S[ốo]:?\s*\d{1,5}\s*/\s*[A-Z\- ]{2,}"
                },
                {
                    "clause": "DOC-03",
                    "title": "Ngày ban hành",
                    "description": "Tài liệu phải ghi rõ ngày ban hành để xác định tính hiệu lực.",
                    "example": "ngày 11 tháng 11 năm 2024",
                    "pattern_hint": r"ngày\s+\d{1,2}\s+tháng\s+\d{1,2}\s+năm\s+\d{4}"
                },
                {
                    "clause": "DOC-04",
                    "title": "Người phê duyệt",
                    "description": "Tài liệu phải thể hiện người có thẩm quyền đã xem xét và phê duyệt.",
                    "example": "Hiệu trưởng: Nguyễn Văn A",
                    "pattern_hint": r"(phê duyệt|ký duyệt|Hiệu trưởng|Giám đốc).*?:.*"
                },
                {
                    "clause": "DOC-05",
                    "title": "Phiên bản",
                    "description": "Phải có chỉ số phiên bản hoặc số hiệu để quản lý các lần sửa đổi.",
                    "example": "Phiên bản: 01 – Ngày cập nhật: 01/01/2024",
                    "pattern_hint": r"(phiên bản|ver|v\.|số hiệu).*?:?.*"
                },
                {
                    "clause": "DOC-06",
                    "title": "Người soạn thảo",
                    "description": "Tài liệu nên ghi rõ người soạn thảo để truy vết nguồn gốc và trách nhiệm.",
                    "example": "Người soạn thảo: Trần Thị B",
                    "pattern_hint": r"(Người soạn thảo|Biên soạn|Tác giả).*?:.*"
                },
                {
                    "clause": "DOC-07",
                    "title": "Phạm vi áp dụng",
                    "description": "Tài liệu cần mô tả rõ phạm vi áp dụng để người dùng hiểu đúng đối tượng điều chỉnh.",
                    "example": "Áp dụng cho toàn bộ giáo viên trường Tiểu học Hương Sơn",
                    "pattern_hint": r"(Phạm vi áp dụng|Áp dụng cho|Áp dụng từ).*"
                },
                {
                    "clause": "DOC-08",
                    "title": "Biểu mẫu đính kèm",
                    "description": "Tài liệu nên liệt kê các biểu mẫu liên quan, nếu có, để người dùng dễ thực hiện.",
                    "example": "Biểu mẫu BM-01: Phiếu đánh giá giáo viên",
                    "pattern_hint": r"(Biểu mẫu|Phụ lục|BM-\d{2}).*"
                }
            ],
            "9.2": [
                {
                    "clause": "AUD-01",
                    "title": "Lập kế hoạch đánh giá nội bộ",
                    "description": "Tổ chức phải lập kế hoạch đánh giá nội bộ định kỳ, đảm bảo bao phủ toàn bộ hệ thống."
                },
                {
                    "clause": "AUD-02",
                    "title": "Tiêu chí và phạm vi đánh giá",
                    "description": "Mỗi cuộc đánh giá phải xác định rõ tiêu chí, phạm vi và phương pháp tiến hành."
                },
                {
                    "clause": "AUD-03",
                    "title": "Chọn đánh giá viên phù hợp",
                    "description": "Người đánh giá phải độc lập với quá trình được đánh giá và có đủ năng lực."
                },
                {
                    "clause": "AUD-04",
                    "title": "Ghi nhận kết quả đánh giá",
                    "description": "Kết quả đánh giá phải được ghi lại đầy đủ và minh bạch để đối chiếu và cải tiến."
                },
                {
                    "clause": "AUD-05",
                    "title": "Hành động khắc phục",
                    "description": "Các phát hiện cần hành động khắc phục phải được xử lý kịp thời và theo dõi hiệu quả."
                }
            ]
        }

        # Gộp tất cả vào 'all'
        self.clause_library["all"] = self.clause_library["7.5"] + self.clause_library["9.2"]

    def extract_text_from_pdf(self, file: BytesIO) -> str:
        reader = PdfReader(file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text.strip()

    def retrieve_clauses(self, document_text: str, clause_group: str = "7.5", top_k: int = 10) -> List[Dict]:
        all_clauses = self.clause_library.get(clause_group, [])
        return all_clauses[:top_k]

    def build_prompt(self, content: str, clauses: List[Dict]) -> str:
        clause_list = "\n".join([f'{c["clause"]}: {c["title"]}' for c in clauses])
        return f"""
Bạn là chuyên gia đánh giá tài liệu theo ISO 9001:2015.

Nội dung tài liệu:
\"\"\"
{content[:3000]}
\"\"\"

Tiêu chí đánh giá:
{clause_list}

Đánh giá tài liệu, đảm bảo không bỏ sót các tiêu chí, trả về JSON với định dạng:
{{
  "compliance_results": {{
    "DOC-01": {{
      "title": "...",
      "score": 100,
      "status": "Đạt" hoặc "Không đạt",
      "evidences": ["..."]
    }},
    ...
  }}
}}
"""

    def call_gemini(self, prompt: str) -> dict:
        response = self.model.generate_content(prompt)
        raw_text = response.text.strip()
        try:
            if raw_text.startswith("```json") or raw_text.startswith("```"):
              raw_text = re.sub(r"^```(?:json)?", "", raw_text)
              raw_text = re.sub(r"```$", "", raw_text)
              raw_text = raw_text.strip()

            return json.loads(raw_text)
        except Exception as e:
            raise ValueError(f"Lỗi xử lý Gemini: {e}\nPhản hồi:\n{response.text}")

    def process_document(self, file_stream: BytesIO) -> dict:
        content = self.extract_text_from_pdf(file_stream)
        clauses = self.retrieve_clauses(content)
        prompt = self.build_prompt(content, clauses)
        result = self.call_gemini(prompt)
        return result

    def generate_compliance_summary(self, compliance_results: dict) -> Tuple[float, List[Tuple[str, dict]]]:
        total, count = 0, 0
        results = []
        for clause, result in compliance_results.items():
            total += result["score"]
            count += 1
            results.append((clause, result))
        avg_score = total / count if count else 0
        return avg_score, results

# Finetuned Chatbot

In [ ]:
class ISO9001ChatBot:
    def __init__(self, model_path: str):
        self.base_model = "Viet-Mistral/Vistral-7B-Chat"
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        base = AutoModelForCausalLM.from_pretrained(
            self.base_model,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )

        from peft import PeftModel
        self.model = PeftModel.from_pretrained(base, model_path)
        self.model.eval()

    def _classify_question(self, question: str) -> str:
        # Định nghĩa nội dung trao đổi theo format chat
        messages = [
            {
                "role": "system",
                "content": "Bạn là một trợ lý chuyên đánh giá câu hỏi người dùng về quản lý chất lượng (QMS)."
            },
            {
                "role": "user",
                "content": f"""Phân loại câu hỏi sau thành một trong hai loại:

    1. document_search — nếu người dùng muốn tìm, xem hoặc yêu cầu một tài liệu cụ thể (như biểu mẫu, file, SOP, quy trình...).
    2. normal_question — nếu người dùng đang hỏi về kiến thức, khái niệm, yêu cầu của ISO 9001:2015.

    Chỉ trả lời duy nhất một từ: document_search hoặc normal_question.

    Câu hỏi: "{question}"
    Loại:"""
            }
        ]

        # Áp dụng chat template của model
        prompt = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Tokenize prompt theo chuẩn của Vistral
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=10,
                do_sample=False,  # sinh chắc chắn
                pad_token_id=self.tokenizer.eos_token_id
            )

        result = self.tokenizer.decode(
            output[0][inputs["input_ids"].shape[1]:],
            skip_special_tokens=True
        ).strip().lower()

        # Phân loại kết quả
        if "document_search" in result:
            return "document_search"
        elif "normal_question" in result:
            return "normal_question"
        else:
            return "unknown"

    def _extract_keywords(self, question: str) -> List[str]:
        messages = [
            {
                "role": "system",
                "content": "Bạn là trợ lý QMS. Nhiệm vụ của bạn là trích xuất từ khóa tìm kiếm từ câu hỏi tiếng Việt."
            },
            {
                "role": "user",
                "content": f"""Câu hỏi: "{question}"

        Hãy trích xuất các từ khóa chính (dưới dạng tiếng Việt), phân cách bằng dấu phẩy.
        Không dịch sang tiếng Anh. Chỉ liệt kê từ khóa, không giải thích.
        """
            }
        ]

        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        print(prompt)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=30,
                do_sample=False,
                pad_token_id=self.tokenizer.eos_token_id
            )

        result = self.tokenizer.decode(output[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
        return [kw.strip() for kw in result.split(",") if kw.strip()]

    def _chat_sync(self, question: str) -> str:
        messages = [
            {
                "role": "system",
                "content": "Bạn là trợ lý ISO 9001:2015. Dựa trên ngữ cảnh cuộc trò chuyện, hãy trả lời câu hỏi mới nhất một cách chi tiết, chính xác và thực tế."
            }
        ] + [
            {
                "role": "user",
                "content": f"""{question}

    Trả lời: """
            }
        ]

        # Tạo prompt chat đúng định dạng
        prompt = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        print(prompt)
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048, padding=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=300,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        return self.tokenizer.decode(output[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    async def chat(self, question: str) -> str:
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, self._chat_sync, question)


# API

In [ ]:
# Danh sách từ khóa QMS
qms_keywords = [
    "iso 9001", "hệ thống quản lý chất lượng", "qms", "tiêu chuẩn iso", "iso",
    "đánh giá nội bộ", "đánh giá chất lượng", "kiểm tra chất lượng", "cải tiến liên tục",
    "kiểm soát chất lượng", "đào tạo chất lượng", "rà soát quản lý", "phân tích nguyên nhân",
    "hành động khắc phục", "hành động phòng ngừa", "báo cáo không phù hợp",
    "quản lý rủi ro", "giám sát và đo lường", "Điều khoản", "Sổ tay hướng dẫn",
    "quản lý tài liệu", "kiểm soát tài liệu", "lưu trữ tài liệu", "phiên bản tài liệu",
    "bản phát hành", "phê duyệt tài liệu", "thay đổi tài liệu", "phân phối tài liệu",
    "thu hồi tài liệu", "tài liệu hướng dẫn", "biểu mẫu", "quy trình", "quy định",
    "hồ sơ chất lượng", "tài liệu viết tay", "tài liệu điện tử", "hướng dẫn công việc",
    "kiểm soát thay đổi", "ghi nhận tài liệu", "truy cập tài liệu", "hướng dẫn",
    "trách nhiệm", "quyền hạn", "người phụ trách", "bộ phận quản lý chất lượng",
    "phân công nhiệm vụ", "người soạn thảo", "người phê duyệt", "người sử dụng tài liệu",
    "đào tạo nhân viên", "giám sát", "điều khoản", "tài liệu", "hướng dẫn",
    "đánh giá tài liệu", "kiểm tra hiệu quả", "đo lường hiệu quả", "phản hồi",
    "cải tiến tài liệu", "kiểm toán nội bộ", "báo cáo kiểm toán", "khắc phục sai sót",
    "rủi ro", "quản lý rủi ro", "biện pháp phòng ngừa", "hành động khắc phục",
    "điều tra nguyên nhân", "xử lý sự cố", "ncr", "hướng dẫn", "viết", "phiên bản", "loại",
    "hệ thống iso", "quy trình làm việc", "tiêu chí chất lượng", "bảng kiểm tra",
    "hồ sơ kiểm tra", "biên bản họp", "chứng nhận iso", "mẫu biểu", "biểu mẫu", "quy trình",
    "quy định", "sổ tay chất lượng", "sổ tay hướng dẫn", "mục", "nội dung",
    "hồ sơ chất lượng", "tài liệu viết tay", "tài liệu điện tử", "hướng dẫn công việc",
    "kiểm soát thay đổi", "ghi nhận tài liệu", "truy cập tài liệu"
]

def preprocess_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    # Loại bỏ dấu câu
    text = re.sub(r'[^\w\s]', ' ', text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def classify_question_keyword_based(question):
    question_processed = preprocess_text(question)
    for keyword in qms_keywords:
        if keyword in question_processed:
            return "related"
    return "unrelated"

# Ví dụ test
questions = [
    "Tổ chức có nên lưu trữ tài liệu giấy không?",
    "AI sẽ thay thế con người trong tương lai?",
    "Quy trình phê duyệt tài liệu diễn ra như thế nào?",
    "Hôm nay thời tiết thế nào?",
    "Làm sao để kiểm soát thay đổi tài liệu hiệu quả?",
    "giải thích về NCR"
]

for q in questions:
    result = classify_question_keyword_based(q)
    print(f"Câu hỏi: {q}\nPhân loại: {result}\n")

Câu hỏi: Tổ chức có nên lưu trữ tài liệu giấy không?
Phân loại: related

Câu hỏi: AI sẽ thay thế con người trong tương lai?
Phân loại: unrelated

Câu hỏi: Quy trình phê duyệt tài liệu diễn ra như thế nào?
Phân loại: related

Câu hỏi: Hôm nay thời tiết thế nào?
Phân loại: unrelated

Câu hỏi: Làm sao để kiểm soát thay đổi tài liệu hiệu quả?
Phân loại: related

Câu hỏi: giải thích về NCR
Phân loại: related



In [ ]:
# chatbot = ISO9001ChatBot("./iso9001-fine-tuned-model/content/iso9001-fine-tuned-model")
rag_chatbot = ISO9001RAGChatbot(gemini_api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
# rag_chatbot.test()

In [ ]:
# Cho phép gọi từ frontend khác domain nếu cần
app = FastAPI(title="ISO 9001 Chatbot API")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ClauseResult(BaseModel):
    clause: str
    title: str
    status: str
    score: float
    evidences: List[str] = []

class AnalyzeResponse(BaseModel):
    score: float
    clause_results: List[ClauseResult]

class KeywordResponse(BaseModel):
    keywords: List[str]

class ChatRequest(BaseModel):
    question: str

class ChatResponse(BaseModel):
    response: str

class ChatMessage(BaseModel):
    role: str
    content: str

class ChatFullRequest(BaseModel):
    question: str
    history_messages: List[ChatMessage]

@app.get("/")
async def root():
    return {"message": "ISO 9001 Chatbot API is running"}

@app.post("/get-label", response_model=ChatResponse)
async def chat(req: ChatRequest):
    if not req.question.strip():
        raise HTTPException(status_code=400, detail="Question is required.")

    try:
        print("Calling chatbot...")
        if(classify_question_keyword_based(req.question) == "unrelated"):
            response = "unknown"
        else:
            response = chatbot._classify_question(req.question)

        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/extract-keywords", response_model=KeywordResponse)
async def extract_keywords(req: ChatRequest):
    if not req.question.strip():
        raise HTTPException(status_code=400, detail="Question is required.")

    try:
        keywords = chatbot._extract_keywords(req.question)
        return {"keywords": keywords}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/chat", response_model=ChatResponse)
async def chat(req: ChatRequest):
    if not req.question.strip():
        raise HTTPException(status_code=400, detail="Question is required.")

    try:
        response = chatbot._chat_sync(req.question)
        return {"response": response}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze(document: UploadFile = File(...)):
    try:
        # Đọc nội dung file PDF
        content = await document.read()
        file_stream = BytesIO(content)
        file_stream.name = document.filename

        # Gọi xử lý tài liệu bằng Gemini + RAG
        doc_results = rag_chatbot.process_document(file_stream)

        # Sinh điểm trung bình và kết quả từng tiêu chí
        score, clause_results_raw = rag_chatbot.generate_compliance_summary(
            doc_results["compliance_results"]
        )

        # Chuyển thành danh sách ClauseResult
        clause_results = [
            ClauseResult(
                clause=clause,
                title=data["title"],
                status=data["status"],
                score=data["score"],
                evidences=data.get("evidences", [])
            )
            for clause, data in clause_results_raw
        ]

        return AnalyzeResponse(score=score, clause_results=clause_results)

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Lỗi xử lý tài liệu: {e}")

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-93' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

# Run

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from google.colab import userdata

# Áp dụng patch cho asyncio để chạy trong notebook
nest_asyncio.apply()

ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

# Mở cổng 8000 ra internet
public_url = ngrok.connect(addr=8000, domain="ample-wildly-parrot.ngrok-free.app")
print("Public URL:", public_url)

# Chạy server
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://ample-wildly-parrot.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2814]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "GET /openapi.json HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-71' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     2001:ee0:4f0e:f780:535:3ca4:d709:e332:0 - "POST /analyze HTTP/1.1"